In [4]:
import csv
import pandas as pd
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

############################################################################################
#Initial variables
#This part of the code is used to set the inital parameters for the web crawler
############################################################################################
START_PAGE = 2
END_PAGE = 5
PROJECT_ABBR = "cedars"
PROJECT_NAME = "Project 5"
URL = "https://web2.cedars.hku.hk:4181/jobs/searchtutor.php?&page=1"

#For login
USERNAME = ''
PASSWORD = ''

#Main window
driver =webdriver.Firefox(executable_path=r'D:\git\FINA2390-Programs\geckodriver.exe')
driver.get(URL)

#Secondary window
driver2 = webdriver.Firefox(executable_path=r'D:\git\FINA2390-Programs\geckodriver.exe')

df_columns = ['Date',
              'Year of Study',
              'Subject',
              'Sex',
              'Location',
              'Time',
              'Salary',
              'Remarks']


df_etf = pd.DataFrame(columns=df_columns)

############################################################################################
#Class
#This particular class to define the conditions for explict wait
#The condition states that the crawler will jump to next page if and only if the webpage can find the text presented
############################################################################################
class text_appear(object):
  def __init__(self, locator1, locator2, locator3):
    self.locator1 = locator1
    self.locator2 = locator2
    self.locator3 = locator3

  def __call__(self, driver):
    element1 = driver.find_element(*self.locator1) 
    element2 = driver.find_element(*self.locator2)
    element3 = driver.find_element(*self.locator3)
    if len(element1.text) != 0 and len(element2.text) !=0 and len(element3.text) !=0:
        return True
    else:
        return False
def driver_login():
    driver.find_element_by_xpath('/html/body/div[3]/a[1]').click
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="username"]').send_keys(USERNAME)
    driver.fine_element_by_xpath('//*[@id="password"]').send_keys(PASSWORD)
    
def go_to_page(page):
    if page != 1:
        print 'GOING TO PAGE' + str(page)
        driver.get('https://web2.cedars.hku.hk:4181/jobs/searchtutor.php?&page='+str(page))
        print 'ARRIVED PAGE' + str(page)
    else:
        return

def save_file(state, CURRENT_PAGE):
    if state == 'backup':
        filename = '['+ PROJECT_ABBR +'][Intrim Backup][' + str(CURRENT_PAGE) + ']' + PROJECT_NAME + ' PAGE ' + str(START_PAGE) + ' to PAGE ' + str(CURRENT_PAGE) + ' Records.csv'
        df_etf.to_csv(filename, index=False,  encoding='utf_8_sig')
        print '[Intrim Backup]Saved First '+ str(CURRENT_PAGE) + 'Pages Records to CSV'
    elif state == 'all':
        filename = '['+ PROJECT_ABBR +']'+ PROJECT_NAME +' PAGE ' + str(START_PAGE) + 'to PAGE ' + str(END_PAGE) + 'Records.csv'
        df_etf.to_csv(filename, index=False, encoding='utf_8_sig')
        print 'Exported to csv'
        
    
def dig(link, row_index):
    
    #Open a new window
    driver2.get(link.get_attribute('href'))
    time.sleep(1)
    
    xsex = '//*[@id="content"]/div[4]/span/table/tbody/tr/td[3]'
    xyof = '//*[@id="content"]/div[4]/span/table/tbody/tr/td[2]'
    xsubject = '//*[@id="content"]/div[4]/span/table/tbody/tr/td[4]/ul/li'
    xtime = '//*[@id="content"]/div[5]/span'
    xsalary = '//*[@id="content"]/div[11]/span'
    xremarks = '//*[@id="content"]/div[11]/span'
    
    #find fields on the new driver
    time.sleep(1)
    if len(driver2.find_element_by_xpath(xsubject)) != 0:
        delimiter = ','
        subjects = []
        for subject in driver2.find_element_by_xpath(xsubject):
            subjects.append(subject)
            
        df_etf.loc[row_index, 'Subject'] = delimiter.join(subjects)
    if len(driver2.find_element_by_xpath(xtime)) != 0:
        df_etf.loc[row_index, 'Time']    = driver2.find_element_by_xpath(xtime).text + ' Sessions per week'
    if len(driver2.find_element_by_xpath(xsalary)) != 0:    
        df_etf.loc[row_index, 'Salary']  = driver2.find_element_by_xpath(xsalary).text
    if len(driver2.find_element_by_xpath(xremarks)) != 0:
        df_etf.loc[row_index, 'Remarks']  = driver2.find_element_by_xpath(xremarks).text
    if len(driver2.find_element_by_xpath(xyos)) != 0:
        df_etf.loc[row_index, 'Year of Study']  = driver2.find_element_by_xpath(xyos).text
    if len(driver2.find_element_by_xpath(xsex))!= 0:
        df_etf.loc[row_index, 'Sex']  = driver2.find_element_by_xpath(xsex).text
    
                                                

############################################################################################
#Main
#This part is the main function of the program
############################################################################################
def main():
    time.sleep(3)
    driver_login()
                                         
    go_to_page(START_PAGE)
    time.sleep(1)
    print 'Start at PAGE'+ str(START_PAGE)
    
    n = 1
    for CURRENT_PAGE in range(START_PAGE,END_PAGE+1):
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' START--------------------'
        START_TIMESTAMP = time.time()
        
        
        for row in driver.find_elements_by_xpath('//*[@id="search_tutors"]/tbody/tr'):
            df_etf.loc[n, 'Location']         = row.find_element_by_xpath('td[2]/a').text
            df_etf.loc[n, 'Date'] = row.find_element_by_xpath('td[4]/a').text

            link = row.find_element_by_xpath('td[1]/a')
            
            #open the univesity profile to get other fields
            #dig(link, n)
            
            print '--------------------ROW ' + str(n)+ ' FINISHED--------------------'
            n = n + 1
            
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' FINISH--------------------'
        END_TIMESTAMP = time.time()
        print '-TIME USED:' + str(START_TIMESTAMP-END_TIMESTAMP) + '-'
        
        #store every first 5,10,15... pages to CSV for backup purpose
        if(CURRENT_PAGE % 5) == 0:
            save_file('backup', CURRENT_PAGE)
            
        go_to_page(CURRENT_PAGE + 1)
        
    #set Source as QS for every row as this is crawl from QS
    
    #export the file to CSV
    save_file('all', CURRENT_PAGE)
    
    #close the driver
    driver.quit()
    
if __name__ == "__main__":
    main()



NoSuchElementException: Message: Unable to locate element: //*[@id="username"]


In [11]:
df_etf


,Date,Year of Study,Subject,Sex,Location,Time,Salary
1,2017-12-21,F.1,"全科,英文,數學",女,香港仔海怡半島,"星期四/五4:30-6:00pm,1.5小時",$120/小時
2,2017-12-21,P.1,英文,男,沙田水泉澳,"星期二4:00-6:00pm,四6:30-8:30pm,2小時(下星期補三堂,補星期二三五4...",$100/小時
3,2017-12-21,P.6,全科,女,元朗臻柏,"星期二,四 6:00-7:30pm,1.5小時",$100/小時
4,2017-12-21,P.6,全科,其它,北角渣華大廈,"星期二四五,4:00-7:00pm,3小時",$80/小時
5,2017-12-21,P.2,全科(高主教),男,銅鑼灣大坑道光明台,"星期一二4:00pm后,星期六,早上,1.5小時",$100/小時
6,2017-12-21,F.1,"中文,數學(英中)",男,元朗大棠路德宗苑,"一星期2-3天,6:00pm,1.5小時",$100/小時
7,2017-12-21,F.5,Geog,女,油塘油麗村,"一星期 1 天, 1.5小時",$150/小時
8,2017-12-21,F.3,中英數(中中),女,大圍顯田村,"星期一四4:30pm,六日4:00pm,其中2天,1.5小時",$120/小時
9,2017-12-21,P.4,"全科,中文,英文,數學",男,青衣長青邨,星期一三5:00-6:30;星期五 4:00-5:30pm,$90/小時
10,2017-12-21,P.3,全科,男,荔枝角碧海藍天,"星期一至四,8:00-9:30pm,1.5小時",$100/小時
